In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os
import errno
import string
from nltk.corpus import reuters
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.text import TextCollection
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score
from sklearn.metrics import accuracy_score,roc_auc_score,mean_absolute_error,cohen_kappa_score
from sklearn.preprocessing import label_binarize,LabelEncoder
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold,StratifiedKFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import load_iris
import warnings
import random
from statistics import mode
from sklearn.decomposition import PCA
random.seed(0)
warnings.filterwarnings('ignore')

def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-2]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels

def Cosine(a, b):#distance
    return distance.cosine(a,b)

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)

def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=[11],train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    if std==[]:
        x.field_names = ["Split",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["Split",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=np.add(train_time,test_time)
    for i in range(0,len(Arr)):
        print(train_time[i])
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if std==[]:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if std==[]:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)
####main Di score computation function############### 
def findDI(train_data,train_labels,metric,k):
    # get all clases from trainig data
    clss=set(train_labels)
    #distance metric
    func=globals()[metric]
    #radius intilization as empty dictionary
    radius={}
    th=0.5
    # iterate over all the classes
    for cl in clss:
        #select indices where training data belong to class c
        indices=[i for i in range(0,len(train_labels)) if train_labels[i]==cl]
        #select training data belonging to class c
        elements=train_data[indices,:]
        #computer pairwise distnace of training data points
        distMat=pairwise_distances(elements,elements,metric=func)
        #computer mean distnace value for class c and set it as radius for class c
        radius[cl]=np.sum(distMat)/(distMat.shape[0]+distMat.shape[1])
    weights=[]
    # build KNN classifier with k neighbors
    classifier = NearestNeighbors(n_neighbors=k,metric=func)
    # if k is greater than no of data points in class c, set k as total data point in class c for that class
    if k>len(train_labels):
         classifier = NearestNeighbors(n_neighbors=len(train_labels),metric=func)
    #train classifier    
    classifier.fit(train_data,train_labels)
    # get k neighbors for all training data points
    distances,neighVals=classifier.kneighbors(train_data,return_distance=True)
    print(neighVals.shape)
    # iterate over all training point
    for i in range(0,train_data.shape[0]):
        #get radius of class of training point
        r=radius[train_labels[i]]
        ID=0
        #get neighvors of training point i
        neighs=np.array(neighVals[i,:],dtype=int)
        #get neighbor labels of training point i
        neighLabels = [train_labels[i] for i in neighs]
        #initialize variables
        val=0
        c=1
        #iterate over all neighbors
        for nn,neigh in enumerate(neighLabels):
            #if distance of neighbor and data point is less than radius
            if distances[i,nn]<r:
                #incremnet c
                c=c+1
                # if neighbors has same label as training point i
                if neigh==train_labels[i]:
                    #increment val
                    val=val+1
        #set score as val/c
        ID=(val/c)
        #append it to DI weights
        weights.append(ID)
    #return Di Weights
    return weights
####End of main Di score computation function############### 
def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    kappa=[]
    averageMeanPrecison=[]
    func=globals()[metric]
    kList=[11]
    theta=1
    var=np.zeros(train_data.shape[1])
    train_time=[]
    test_time=[]
    M=5
    alpha=0.5
    ###################start of main code############
    #iterate over k
    for k in kList:
        time1=time.time()
        # call main function to compute Di weights
        weights=findDI(train_data,train_labels,metric,k)
        #define weight function to use it in weighted KNN
        def weighted(distances):
            return weights
        #build weighted KNN classsifier using DI weights    
        classifier = KNeighborsClassifier(n_neighbors=k,metric=func,weights=locals()['weighted'])
        # if k is greater than no of data points, set k as total data point 
        if k>len(train_labels):
             classifier = KNeighborsClassifier(n_neighbors=len(train_labels),metric=func,weights=locals()['weighted'])
        # train classifier
        classifier.fit(train_data,train_labels)
        # get neighbors for test points
        distances,neighVals=classifier.kneighbors(test_data,return_distance=True)
        print("test",distances.shape)
        pred=[]
        time2=time.time()
        # iterate over test data points
        for i in range(0,test_data.shape[0]):
            # get neighbors of test point
            neighs=np.array(neighVals[i,:],dtype=int)
            # get labels of neighbors of test point
            neighLabels = [train_labels[i] for i in neighs]
            # select unique lables from neighbors
            dLabels=set(neighLabels)
            #initialize variables
            weightSum=0
            maxweight=0
            predLabel=''
            # iterate over unique lables from neighbors
            for lbl in dLabels:
                weightSum=0
               # compute sumof weights of each class
                weightSum=np.sum([weights[ii] for j,ii in enumerate(neighs) if train_labels[ii]==lbl])
                # assign label of class with maximum weights as prediction
                if weightSum>=maxweight:
                    maxweight=weightSum
                    predLabel=lbl
            #append to prediction
            pred.append(predLabel)  
        y_pred.append(pred)
        ###################End of main code############
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
 
   
    class_dict={}
    classes=list(set(labels))
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)
        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
        
        kappa.append(cohen_kappa_score(test_labels, y_pred[i]))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
            
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)



    
    '''
    #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
    '''
    accuracy_avg.extend(accuracy)
    precision_avg.extend(precision)
    recall_avg.extend(Recall)
    macroFMeasure_avg.extend(fMeasure)
    mprecision_avg.extend(mprecision)
    mrecall_avg.extend(mRecall)
    mFMeasure_avg.extend(mfMeasure)
    roc_avg.extend(roc)
    AMP_avg.extend(averageMeanPrecison)
    mae_avg.extend(mae)
    avg_test_time.extend(test_time)
    avg_train_time.extend(train_time)
    
import glob
datasets=[]
for file_name in glob.glob("Selected Datasets\\"+'*.csv'):
    file=os.path.split(file_name)[1]
    datasets.append(os.path.splitext(file)[0])

    
for dataset in datasets:
    fname="Selected Datasets\\"+dataset
    print(fname.encode())
    arr,labels=loadNonIRDatasets(fname)
    arr=np.array(arr)
    labels=np.array(labels)
    labels=list(labels)
    le = LabelEncoder()
    labels=le.fit_transform(labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    global w
    w=0
    global Weightsknn
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "EPrkNN\\k=11\\Selected Datasets"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_EPrKNN_k=11.txt'
        tables = open(fname, 'wb')
        k_splits=10
        accuracy_avg=[]
        precision_avg=[]
        recall_avg=[]
        macroFMeasure_avg=[]
        mprecision_avg=[]
        mrecall_avg=[]
        mFMeasure_avg=[]
        roc_avg=[]
        AMP_avg=[]
        mae_avg=[]
        avg_test_time=[]
        avg_train_time=[]
        kf = StratifiedKFold(n_splits=k_splits)
        for trn_ind, tst_ind in kf.split(arr,labels):
            xTrain, xTest = arr[trn_ind], arr[tst_ind]
            yTrain, yTest = labels[trn_ind], labels[tst_ind]
            train_data=np.array(xTrain)
            test_data=np.array(xTest)
            train_labels=np.array(yTrain)
            test_labels=np.array(yTest)
            allData=arr
            termOcc=[]
            docOcc=[]
            tables.write("\n*************".encode())
            tables.write((" Split\t"+str(n_split)).encode())
            tables.write(" ***********\n".encode())
            n_split=n_split+1
            k=1
            print("###############")
            Weights=np.ones(train_data.shape[0])
            classify(k=k,metric=met,termOccurance=termOcc,docOccurance=docOcc)
        list1=list(range(1,11))
        tables.write("\n************* Average Results ***********\n".encode())
        
        acc_std=[]#np.std(np.array(accuracy_avg))
        #accuracy_avg=np.mean(np.array(accuracy_avg),axis=0)
        
        pr_std=[]#np.std(np.array(precision_avg))
        #precision_avg=np.mean(np.array(precision_avg),axis=0)
        
        mpr_std=[]#np.std(np.array(mprecision_avg),axis=0)
        #mprecision_avg=np.mean(np.array(mprecision_avg),axis=0)
        
        rcl_std=[]#np.std(np.array(recall_avg))
        #recall_avg=np.mean(np.array(recall_avg),axis=0)
        
        mrcl_std=[]#np.std(np.array(mrecall_avg))
        #mrecall_avg=np.mean(np.array(mrecall_avg),axis=0)
        
        F_std=[]#std(np.array(macroFMeasure_avg))
        #macroFMeasure_avg=np.mean(np.array(macroFMeasure_avg),axis=0)
        
        mF_std=[]#np.std(np.array(mFMeasure_avg))
        #mFMeasure_avg=np.mean(np.array(mFMeasure_avg),axis=0)
        
        roc_std=[]#np.std(np.array(roc_avg))
        #roc_avg=np.mean(np.array(roc_avg),axis=0)
        
        amp_std=[]#np.std(np.array(AMP_avg))
        #AMP_avg=np.mean(np.array(AMP_avg),axis=0)
        
        mae_std=[]#np.std(np.array(mae_avg))
        #mae_avg=np.mean(np.array(mae_avg),axis=0)
        
        avg_train_time=avg_train_time
        avg_test_time=avg_test_time
        
        #Accuracy
        PrintDetails(metric=met,measure="Accuracy",Arr=accuracy_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=acc_std)
        
        #Precision
        PrintDetails(metric=met,measure="Precision",Arr=precision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=pr_std)
        
        #Precision
        PrintDetails(metric=met,measure="Micro Precision",Arr=mprecision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mpr_std)


        #Recall
        PrintDetails(metric=met,measure="Recall",Arr=recall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=rcl_std)
        
         #Recall
        PrintDetails(metric=met,measure="Micro Recall",Arr=mrecall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mrcl_std)


        #f measure
        PrintDetails(metric=met,measure="F Measure",Arr=macroFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=F_std)
        
       
       
        #f measure
        PrintDetails(metric=met,measure="Micro F Measure",Arr=mFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mF_std)


        #roc
        PrintDetails(metric=met,measure="ROC",Arr=roc_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=roc_std)

        #average Mean precision
        PrintDetails(metric=met,measure="AMP",Arr=AMP_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=amp_std)
         
        #MAE
        PrintDetails(metric=met,measure="MAE",Arr=mae_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mae_std)
        
        tables.close()
        print("###############")





b'Selected Datasets\\arrhythmia'
###############
(405, 11)
test (46, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
###############
(406, 11)
test (45, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
###############
(406, 11)
test (45, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
###############
(406, 11)
test (45, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
###############
(406, 11)
test (45, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
###############
(406, 11)
test (45, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
###############
(406, 11)
test (45, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
###############
(406, 11)
test (45, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 

###############
(1324, 11)
test (148, 11)
{0: 0, 1: 1, 2: 2}
###############
(1324, 11)
test (148, 11)
{0: 0, 1: 1, 2: 2}
###############
(1325, 11)
test (147, 11)
{0: 0, 1: 1, 2: 2}
###############
(1325, 11)
test (147, 11)
{0: 0, 1: 1, 2: 2}
###############
(1325, 11)
test (147, 11)
{0: 0, 1: 1, 2: 2}
###############
(1325, 11)
test (147, 11)
{0: 0, 1: 1, 2: 2}
###############
(1325, 11)
test (147, 11)
{0: 0, 1: 1, 2: 2}
###############
(1325, 11)
test (147, 11)
{0: 0, 1: 1, 2: 2}
###############
(1325, 11)
test (147, 11)
{0: 0, 1: 1, 2: 2}
###############
(1325, 11)
test (147, 11)
{0: 0, 1: 1, 2: 2}
metric Euclidean
measure Accuracy
6.04999566078186
6.158002853393555
5.98299765586853
6.063999891281128
5.860000848770142
6.10896635055542
6.005999326705933
5.925000905990601
5.870999336242676
6.0650012493133545
+----------+----------+---------------------+-----------------------+---------------------+
|  Split   | Accuracy |      Train Time     |       Test time       |      Total Time 

(620, 11)
test (69, 11)
{0: 0, 1: 1}
###############
(620, 11)
test (69, 11)
{0: 0, 1: 1}
###############
(620, 11)
test (69, 11)
{0: 0, 1: 1}
###############
(620, 11)
test (69, 11)
{0: 0, 1: 1}
###############
(620, 11)
test (69, 11)
{0: 0, 1: 1}
###############
(620, 11)
test (69, 11)
{0: 0, 1: 1}
###############
(620, 11)
test (69, 11)
{0: 0, 1: 1}
###############
(620, 11)
test (69, 11)
{0: 0, 1: 1}
###############
(620, 11)
test (69, 11)
{0: 0, 1: 1}
###############
(621, 11)
test (68, 11)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
1.3679966926574707
1.374000072479248
1.3689815998077393
1.4590060710906982
1.3960003852844238
1.3679993152618408
1.3580002784729004
1.4070219993591309
1.4059998989105225
1.4059996604919434
+----------+----------+----------------------+------------------------+----------------------+
|  Split   | Accuracy |      Train Time      |       Test time        |      Total Time      |
+----------+----------+----------------------+------------------------+--

(302, 11)
test (34, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
###############
(302, 11)
test (34, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
###############
(302, 11)
test (34, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
###############
(302, 11)
test (34, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
###############
(302, 11)
test (34, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
###############
(302, 11)
test (34, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
###############
(303, 11)
test (33, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
###############
(303, 11)
test (33, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
###############
(303, 11)
test (33, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
###############
(303, 11)
test (33, 11)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
metric Euclidean
measure Accuracy
0.21599769592285156
0.21799945831298828
0.21199965476989746
0.2109978199005127
0.20297980308

+----------+--------------+-----------------------+-----------------------+----------------------+
|  Split   | Micro Recall |       Train Time      |       Test time       |      Total Time      |
+----------+--------------+-----------------------+-----------------------+----------------------+
|    1     |    0.8235    |     0:00:00.215998    |     0:00:00.001001    |    0:00:00.216999    |
|    2     |    0.9412    |     0:00:00.217999    |     0:00:00.001000    |    0:00:00.219000    |
|    3     |    0.9412    |     0:00:00.212000    |        0:00:00        |    0:00:00.212000    |
|    4     |    0.9412    |     0:00:00.210998    |     0:00:00.001000    |    0:00:00.211998    |
|    5     |     1.0      |     0:00:00.202980    |     0:00:00.000999    |    0:00:00.203979    |
|    6     |     1.0      |     0:00:00.210000    |        0:00:00        |    0:00:00.210000    |
|    7     |     1.0      |     0:00:00.205002    |        0:00:00        |    0:00:00.205002    |
|    8    

(263, 11)
test (30, 11)
{0: 0, 1: 1}
###############
(263, 11)
test (30, 11)
{0: 0, 1: 1}
###############
(263, 11)
test (30, 11)
{0: 0, 1: 1}
###############
(264, 11)
test (29, 11)
{0: 0, 1: 1}
###############
(264, 11)
test (29, 11)
{0: 0, 1: 1}
###############
(264, 11)
test (29, 11)
{0: 0, 1: 1}
###############
(264, 11)
test (29, 11)
{0: 0, 1: 1}
###############
(264, 11)
test (29, 11)
{0: 0, 1: 1}
###############
(264, 11)
test (29, 11)
{0: 0, 1: 1}
###############
(264, 11)
test (29, 11)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.36100149154663086
0.3699991703033447
0.36496472358703613
0.36986374855041504
0.36699795722961426
0.3749973773956299
0.386000394821167
0.3690001964569092
0.3749985694885254
0.3729989528656006
+----------+----------+---------------------+-----------------------+-----------------------+
|  Split   | Accuracy |      Train Time     |       Test time       |       Total Time      |
+----------+----------+---------------------+-----------------------+--

(315, 11)
test (36, 11)
{0: 0, 1: 1}
###############
(316, 11)
test (35, 11)
{0: 0, 1: 1}
###############
(316, 11)
test (35, 11)
{0: 0, 1: 1}
###############
(316, 11)
test (35, 11)
{0: 0, 1: 1}
###############
(316, 11)
test (35, 11)
{0: 0, 1: 1}
###############
(316, 11)
test (35, 11)
{0: 0, 1: 1}
###############
(316, 11)
test (35, 11)
{0: 0, 1: 1}
###############
(316, 11)
test (35, 11)
{0: 0, 1: 1}
###############
(316, 11)
test (35, 11)
{0: 0, 1: 1}
###############
(316, 11)
test (35, 11)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.5239977836608887
0.5290021896362305
0.5199964046478271
0.5279982089996338
0.5309998989105225
0.5229992866516113
0.5169928073883057
0.5350003242492676
0.566997766494751
0.5479662418365479
+----------+----------+---------------------+------------------------+----------------------+
|  Split   | Accuracy |      Train Time     |       Test time        |      Total Time      |
+----------+----------+---------------------+------------------------+-----

In [2]:
"test"

'test'